In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import suspect
import multiprocessing as mp
plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
os.getcwd()

## 1. Preprocess Subtracted

In [ ]:
# Macromolecule files are the TWIX (.dat) files with ad-special

In [ ]:
mm_dats = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_3T/rawdata/sub-*/*ad-special*_765*35*.dat')
#mm_dats = sorted(mm_dats, key=lambda x: x.split('.')[1].split('_')[-2])
mm_dats

In [ ]:
mm_dats.pop(-1)

In [ ]:
mm_dats

In [ ]:
mm_dats.pop(-3)
mm_dats.pop(-3)
mm_dats += ['/home/orco/data/MacroMols/MRS/MacroMols_3T/rawdata/sub-001/sub-001_nuc-1H_loc-pcc_spec-ad-special_TI1_765_TI2_35.dat']

In [ ]:
mm_dats.pop(3)

In [ ]:
specs = [i.replace('spec-ad-special_TI1_765_TI2_35.dat', 'spec-rm-special.dat') for i in mm_dats]


In [ ]:
specs

In [ ]:
[os.path.isfile(i) for i in specs] == [True]*len(specs)

In [ ]:
# take the 701 and average it
s1 = ['/home/orco/data/MacroMols/MRS/MacroMols_3T/rawdata/sub-001/sub-001_nuc-1H_loc-pcc_spec-ad-special_TI1_765_TI2_35.dat',
      '/home/orco/data/MacroMols/MRS/MacroMols_3T/rawdata/sub-001/sub-001_nuc-1H_loc-pcc_spec-ad-special_TI1_765_TI2_35_2.dat']

In [ ]:
s1

In [ ]:
s11 = [suspect.io.load_twix(i) for i in s1]

In [ ]:
len(s11)

In [ ]:
s111 = np.append(s11[0], s11[1], axis=0)

In [ ]:
s111.shape

In [ ]:
s111 = s11[0].inherit(s111)

In [ ]:
water = [i.replace('spec-ad-special_TI1_765_TI2_35.dat','spec-rm-special-water_wOVS.dat') for i in mm_dats]

In [ ]:
len(specs) == len(water) == len(mm_dats)

In [ ]:
# # No water for 005 ACC
# mm_dats.pop(3)
# specs.pop(3)
# water.pop(3)

## 1.1 Average every two Averages



    Near, Jamie & Harris, Ashley & Juchem, Christoph & Kreis, Roland & Marjańska, Małgorzata & Öz, Gülin & Slotboom, Johannes & Wilson, Martin & Gasparovic, Charles. (2020). Preprocessing, analysis and quantification in single-voxel magnetic resonance spectroscopy: experts' consensus recommendations. NMR in Biomedicine. e4257. 10.1002/nbm.4257.


In [ ]:
%%time
mms = [suspect.io.load_twix(i) for i in mm_dats[:-1]]
specs_data = [suspect.io.load_twix(i) for i in specs]

In [ ]:
mms += [s111]

In [ ]:
len(mms)

In [ ]:
#[i.shape for i in mms]

In [ ]:
waters = [suspect.io.load_twix(i) for i in water]

In [ ]:
len(waters) == len(mms) == len(specs_data)

In [ ]:
plt.plot(specs_data[0][0,0].frequency_axis_ppm(), specs_data[0][4,0].spectrum().real)
plt.plot(specs_data[0][1,0].frequency_axis_ppm(), specs_data[0][5,0].spectrum().real)
plt.xlim([5,0])
plt.axvline(1.5)
plt.axvline(0.55)

In [ ]:
plt.plot(mms[0][0,0].frequency_axis_ppm(), mms[0][4,0].spectrum().real)
plt.plot(mms[0][1,0].frequency_axis_ppm(), mms[0][5,0].spectrum().real)
plt.xlim([5,0])
plt.axvline(1.5)
plt.axvline(0.55)

## 2. Average every odd with every even acquisition

In [ ]:
def mean_everytwo(mm1):
    mm1_everytwo = []
    for i in range(0,mm1.shape[0]-1,2):
        mm1_everytwo.append((mm1[i]+mm1[i+1])/2)
    return mm1.inherit(np.array(mm1_everytwo))


In [ ]:
def _avg_everytwo(arr):
    return 0.5*(arr[0::2,:, :] + arr[1::2,:, :])

In [ ]:
%%time
#avg_everytwo_data = [mean_everytwo(i) for i in mms]
#avg_everytwo_water = [mean_everytwo(i) for i in waters]

In [ ]:
len(mms) == len(waters)

In [ ]:
%%time
avg_everytwo_data = [_avg_everytwo(i) for i in mms]
avg_everytwo_water = [_avg_everytwo(i) for i in waters]
avg_everytwo_specs = [_avg_everytwo(i) for i in specs_data]

In [ ]:
avg_everytwo_data[0].shape

In [ ]:
del(mms)
del(waters)

In [ ]:
plt.plot(avg_everytwo_specs[0][0,0].frequency_axis_ppm(), avg_everytwo_data[0][4,0].spectrum().real)
plt.plot(avg_everytwo_specs[0][1,0].frequency_axis_ppm(), avg_everytwo_data[0][5,0].spectrum().real)
plt.xlim([5,0])
plt.axvline(1.5)
plt.axvline(0.55)

In [ ]:
#[i.shape for i in avg_everytwo]

## 1.2 Coil combination

In [ ]:
def coil_combine(mrs):
    coil_combined = []
    for i in range(mrs.shape[0]):
        weights = suspect.processing.channel_combination.svd_weighting(mrs[i,:,:])
        coil_combined.append(suspect.processing.channel_combination.combine_channels(mrs[i,:,:],weights))
    return mrs.inherit(np.array(coil_combined))

In [ ]:
#Rodgers, C. T., & Robson, M. D. (2010). Receive array magnetic resonance spectroscopy: Whitened singular value decomposition (WSVD) gives optimal Bayesian solution. Magnetic Resonance in Medicine, 63(4), 881–891
def coil_combine(data, wref, noise_points=None):
    if not noise_points:
        noise_points = int(data.shape[-1]/8)
    noise = data[:,:,-noise_points:]
    noise = np.moveaxis(noise, -2, 0).reshape((32, -1))
    white_data = suspect.processing.channel_combination.whiten(data, noise)
    white_wref = suspect.processing.channel_combination.whiten(wref, noise)
    noise = white_data[:, :, -noise_points:]
    channel_weights = suspect.processing.channel_combination.svd_weighting(np.mean(white_wref, axis=0))
    cc_data = suspect.processing.channel_combination.combine_channels(white_data, channel_weights)
    cc_wref = suspect.processing.channel_combination.combine_channels(white_wref, channel_weights)
    return cc_data, cc_wref

In [ ]:
len(avg_everytwo_data)

In [ ]:
len(avg_everytwo_water)

In [ ]:
%%time
cc_data = list()
cc_wref = list()
cc_specs = list()
for n in range(len(avg_everytwo_data)):
    print(n)
    d,w = coil_combine(avg_everytwo_data[n],avg_everytwo_water[n],500)
    s,w = coil_combine(avg_everytwo_specs[n],avg_everytwo_water[n],500)
    cc_data.append(d)
    cc_specs.append(s)
    cc_wref.append(w)

In [ ]:
cc_data[0].shape

In [ ]:
#cc_data = [coil_combine(i) for i in avg_everytwo]

In [ ]:
spectra = cc_data[2].spectrum()
frequency_slice = spectra.slice_ppm(3.5, 0)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 64, 0, 3.5], aspect='auto')

In [ ]:
cc_data[2][0].shape

In [ ]:
for i in cc_data[2][:]:
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real)
    plt.xlim([4.1,0])
    plt.axvline(4.7)
    plt.ylim([-300,800])

In [ ]:
spectra = cc_specs[2].spectrum()
frequency_slice = spectra.slice_ppm(3.5, 0)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 64, 0, 3.5], aspect='auto')

In [ ]:
type(cc_data[0])

In [ ]:
[i.shape for i in cc_data]

In [ ]:
spectra = cc_data[0].spectrum()
frequency_slice = spectra.slice_ppm(3.5, 0)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 64, 0, 3.5], aspect='auto')

In [ ]:
spectra = cc_data[0].spectrum()
frequency_slice = spectra.slice_ppm(3.5, 0)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 128, 0, 3.5], aspect='auto')

## 1.3 Frequency correction

In [ ]:
first = cc_wref[1][0]
plt.plot(first.frequency_axis_ppm(), first.spectrum().real)
plt.xlim([5,0])
plt.axvline(4.7)

In [ ]:
corr = suspect.processing.frequency_correction.residual_water_alignment(first)

In [ ]:
plt.plot(first.frequency_axis_ppm(), first.adjust_frequency(0).spectrum().real)
plt.plot(first.frequency_axis_ppm(), first.adjust_frequency(-corr).spectrum().real)
plt.xlim([5,4])
plt.axvline(4.7)

In [ ]:
first = first.adjust_frequency(-corr)
corr

In [ ]:
w_freq_corr  = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,first) for i in cc_wref]

In [ ]:
first = cc_data[1][6]
plt.plot(first.frequency_axis_ppm(), first.adjust_phase(3).adjust_frequency(0).spectrum().real)
plt.xlim([7,0])
plt.axvline(4.7)
plt.axvline(2.0)
plt.ylim([-300,800])

In [ ]:
first = first.adjust_phase(3).adjust_frequency(0)

In [ ]:
#corr = suspect.processing.frequency_correction.residual_water_alignment(first)
#first = first.adjust_frequency(-corr)
#corr

## 1.4 Coregister to first

In [ ]:
cc_data[0].shape

In [ ]:
[i.shape for i in cc_data]

In [ ]:
cc_data2 = cc_data.copy()

In [ ]:
allcc = np.asarray([np.array(i.spectrum().real) for i in cc_data2])

In [ ]:
allcc.shape

In [ ]:
freq_corr2 = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,first) for i in cc_data]

In [ ]:
first = cc_specs[9][7]
plt.plot(first.frequency_axis_ppm(), first.adjust_phase(0,).adjust_frequency(0).spectrum().real)
plt.xlim([7,0])
plt.axvline(1.99)
plt.axvline(4.7)
plt.axvline(3.027)
plt.axvline(0.94)
plt.axhline(0)

In [ ]:
first = first.adjust_frequency(0)

In [ ]:
freq_corr_spec = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,i[0]) for i in cc_specs]

In [ ]:
# Average Water

In [ ]:
w_avg = [np.mean(i,axis=0) for i in w_freq_corr]

## 1.5 Average

In [ ]:
%%time
averaged_mm = [np.mean(i,axis=0) for i in freq_corr2]
averaged_spec = [np.mean(i,axis=0) for i in freq_corr_spec]

In [ ]:
# ECC

In [ ]:
avg_ecc_mm = list()
for n in range(len(averaged_mm)):
    eddy_current = np.unwrap(np.angle(w_avg[n]))
    ec_smooth = suspect.processing.denoising.sliding_gaussian(eddy_current, 32)
    ecc = np.exp(-1j * ec_smooth)
    ec_data = averaged_mm[n] * ecc
    avg_ecc_mm.append(ec_data)

In [ ]:
for n,i in enumerate(averaged_mm):
    #plt.plot(i.frequency_axis_ppm(), i.spectrum().real, label=n)
    plt.plot(i.frequency_axis_ppm(), avg_ecc_mm[n].spectrum().real, label=n)
plt.xlim([5,0])
plt.ylim([-200,500])

#plt.ylim([-2e-4,5e-4])
plt.axvline(4.7)
plt.legend()

In [ ]:
len(averaged_mm)

In [ ]:
avg_ecc_spec = list()
for n in range(len(averaged_spec)):
    eddy_current = np.unwrap(np.angle(w_avg[n]))
    ec_smooth = suspect.processing.denoising.sliding_gaussian(eddy_current, 32)
    ecc = np.exp(-1j * ec_smooth)
    ec_data = averaged_spec[n] * ecc
    avg_ecc_spec.append(ec_data)

In [ ]:
for n,i in enumerate(avg_ecc_mm[0:1]):
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real, label=n)
plt.xlim([5,0])
plt.ylim([-200,500])
plt.axvline(1.99)
plt.legend()
plt.axhline(0)

In [ ]:
for n,i in enumerate(avg_ecc_spec):
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real, label=n)
plt.xlim([5,0])
#plt.ylim([-200,500])
#plt.axvline(1.99)
plt.legend()
#plt.axhline(0)

# HLSVD

In [ ]:
water_list_mm = list()
dry_list_mm = list()
for spec in avg_ecc_mm:
    components = suspect.processing.water_suppression.hsvd(spec, 30)
    water_components = [component for component in components if component["frequency"] < 60]
    water_fid = spec.inherit(suspect.processing.water_suppression.construct_fid(water_components, spec.time_axis()))
    dry_fid = spec - water_fid
    water_list_mm.append(water_fid)
    dry_list_mm.append(dry_fid)

In [ ]:
for i in water_list_mm:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    plt.ylim([-200,500])
    #plt.axvline(4.7)

In [ ]:
for i in dry_list_mm:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    plt.ylim([-200,500])
    plt.axvline(4.2)

In [ ]:
water_list_spec = list()
dry_list_spec = list()
for spec in avg_ecc_spec:
    components = suspect.processing.water_suppression.hsvd(spec, 30)
    water_components = [component for component in components if component["frequency"] < 60]
    water_fid = spec.inherit(suspect.processing.water_suppression.construct_fid(water_components, spec.time_axis()))
    dry_fid = spec - water_fid
    water_list_spec.append(water_fid)
    dry_list_spec.append(dry_fid)

In [ ]:
for i in water_list_spec:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    plt.ylim([-200,500])
    #plt.axvline(4.7)

In [ ]:
for i in dry_list_spec:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    plt.ylim([-200,500])
    plt.axvline(4.2)

In [ ]:
len(dry_list_mm) == len(dry_list_spec)

In [ ]:
der_path = '/home/orco/data/MacroMols/MRS/MacroMols_3T/derivatives/IMNS'

# HLSVD removing NAAG for MM and subtraction

In [ ]:
i=12
spec = dry_list_mm[i]
components = suspect.processing.water_suppression.hsvd(spec, 20)

In [ ]:
fids = [spec.inherit(suspect.processing.water_suppression.construct_fid([i], spec.time_axis())) for i in components]

In [ ]:
shift = 15
for n,i in enumerate(fids):
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real-n*shift)
    plt.xlim([6,-1])
    #plt.ylim([-200,700])
    #plt.axvline(4.2)
    plt.axvline(3.93)
    #plt.axvline(0.7)
    plt.axvline(2.1)
    #plt.axhline(0)

In [ ]:
artifacts = [components[6]] + [components[16]] #+ [components[8]] #+ [components[8]] + components[16:]
artifacts = spec.inherit(suspect.processing.water_suppression.construct_fid(artifacts, spec.time_axis()))

In [ ]:
#mm_fid = components[2:8] + components[9:17]
#mm_fid = dry_list_mm[i].inherit(suspect.processing.water_suppression.construct_fid(mm_fid, spec.time_axis()))

In [ ]:
i=12
mul_fact = 1.8
ph=-.0
phs=-.0
fr = 5
plt.plot(dry_list_spec[i].frequency_axis_ppm(), dry_list_spec[i].adjust_phase(phs).spectrum().real)
plt.plot(dry_list_mm[i].frequency_axis_ppm(), ((dry_list_mm[i]-artifacts)*mul_fact).adjust_phase(ph).adjust_frequency(fr).spectrum().real)
#plt.plot(dry_list_mm[i].frequency_axis_ppm(), (mm_fid*mul_fact).adjust_phase(ph).adjust_frequency(fr).spectrum().real)
sub = dry_list_spec[i].adjust_phase(phs) - ((dry_list_mm[i]-artifacts)*mul_fact).adjust_phase(ph).adjust_frequency(fr)
#sub = dry_list_spec[i] - (mm_fid*mul_fact).adjust_phase(ph).adjust_frequency(fr)
plt.plot(dry_list_mm[i].frequency_axis_ppm(), sub.spectrum().real)
plt.xlim([6,0])
#plt.ylim([-200,700])
plt.axvline(4.2)
plt.axvline(3.94)
#plt.axvline(0.7)
plt.axhline(0)

In [ ]:
plt.plot(dry_list_mm[i].frequency_axis_ppm(), sub.spectrum().real)
plt.xlim([6,0])
plt.axvline(4.2)
plt.axvline(3.94)
plt.axvline(0.94)
plt.axhline(0)

In [ ]:
# Save
dest_name=specs[i].replace('rawdata/', 'derivatives/IMNS/').replace('.dat', '_subtracted.raw')
os.makedirs(os.path.dirname(dest_name), exist_ok=True)
suspect.io.lcmodel.save_raw(dest_name, sub)

In [ ]:
dest_name

In [ ]:
suspect.io.lcmodel.save_raw

In [ ]:
specs[i]

In [ ]:
mm_dats[i]

In [ ]:
specs

In [ ]:
mm_dats